# Mock Data
- Take the Theta file
- predict spec and phot
- distort spec and phot with a gaussian sigma=phot/snr
- snr has to be smaller than 40

In [23]:
import os
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.cosmology import Planck15 as cosmo
from astropy.table import Table
from astropy.io import fits

from prospect import prospect_args
from prospect.fitting import fit_model
from prospect.io import write_results as writer
from prospect.likelihood import NoiseModel
from prospect.likelihood.kernels import Uncorrelated
from prospect.sources.galaxy_basis import CSPSpecBasis
from prospect.utils.obsutils import fix_obs

import prospect.io.read_results as reader
import toolbox_prospector as toolbox

from sedpy.observate import load_filters

import Build


In [40]:
iot_number = '4'

## Read in all the data

In [41]:
path_wdir   =   "/Users/amanda/Desktop/Paper/technical/"
path_data   =   os.path.join(path_wdir, "data/")
path_plots  =   os.path.join(path_wdir, 'plots/iot_'+iot_number+'/')
path_output =   os.path.join(path_wdir, 'prospector/input_output_test'+iot_number+'/')
path_flury  =   os.path.join(path_data, 'flury/')
path_mock   =   os.path.join(path_data, 'mock/')


#------------------------------------------------------Emission Lines-----------------------------------------------------------------------

el_table  =  "lzlcs_optlines_obs.csv"
el        =   Table.read(os.path.join(path_flury, el_table), format="ascii")

el.add_column([i for i in range(1,67)], name='id', index=0) #uniform reference for name now possible


#-------------------------------------------------------Photometry--------------------------------------------------------------------------

phot_table  =   'GP_Aperture_Matched_Photometry_v0.fits'
phot_cat    =   fits.open(os.path.join(path_flury, phot_table))
phot        =   phot_cat[1].data


#------------------------------------------------------Signal to Noise----------------------------------------------------------------------


snr      = pd.read_csv(path_data + 'snr.csv')
snr_phot = snr['median'][:7] 
snr_el   = snr['median'][7:]


In [42]:
thetas = {}

thetas['parnames']        =  ['logzsol', 'dust2', 'logmass', 'logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios','logsfr_ratios', 'dust_index', 'dust1_fraction', 'igm_factor', 'gas_logz', 'gas_logu', 'frac_obrun']
thetas['thetas_original'] =  [-0.5,  0.6, 10. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. ,  0. , -2. ,  0. ]
thetas['thetas_0']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0. ]
thetas['thetas_1']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.1 ]
thetas['thetas_2']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.3 ]
thetas['thetas_3']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.5 ]
thetas['thetas_4']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.7 ]
thetas['thetas_5']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.9 ]

thetas['thetas_6']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  1. ]
thetas['thetas_7']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.15 ]
thetas['thetas_8']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.2 ]
thetas['thetas_9']        =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.25 ]
thetas['thetas_10']       =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.35 ]
thetas['thetas_11']       =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.4 ]
thetas['thetas_12']       =  [-1.0,  0.6,  8. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. , 1. ,  1. , -1. , -2. ,  0.45 ]

filename = 'mock_thetas'+iot_number+'.pickle'

# open the file in write mode and save the dictionary as a pickle
with open(path_mock+filename, 'wb') as f:
    pickle.dump(thetas, f)

## Build
- build obs with galaxy 3 because it has all the bands and all the emission lines and it doesnt matter which one
- build model
- build sps

In [43]:
thetas.keys()

dict_keys(['parnames', 'thetas_original', 'thetas_0', 'thetas_1', 'thetas_2', 'thetas_3', 'thetas_4', 'thetas_5', 'thetas_6', 'thetas_7', 'thetas_8', 'thetas_9', 'thetas_10', 'thetas_11', 'thetas_12'])

In [44]:
objid   =  3
obs     =  Build.build_obs(objid=objid)
model   =  Build.build_model(objid=objid, fit_el=True)
sps     =  Build.build_sps()

GALAXIE: J011309+000223, ID: 3, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
GALAXIE: J011309+000223, ID: 3, PHOTOMETRY: GP_Aperture_Matched_Photometry_v0.fits, EMISSION_LINES: lzlcs_optlines_obs.csv
SFH template = continuity_sfh
redshift init = 0.3061555
FITTING EL MODEL


## Predict 
- spec and phot predicted for the different thetas, which vary frac_obrun from 0, 0.1, 0.3, 0.5, 0.7, 0.9, 1

In [45]:
spec_0, phot_0, _ = model.predict(thetas['thetas_0'], obs=obs, sps=sps)
spec_1, phot_1, _ = model.predict(thetas['thetas_1'], obs=obs, sps=sps)
spec_2, phot_2, _ = model.predict(thetas['thetas_2'], obs=obs, sps=sps)
spec_3, phot_3, _ = model.predict(thetas['thetas_3'], obs=obs, sps=sps)
spec_4, phot_4, _ = model.predict(thetas['thetas_4'], obs=obs, sps=sps)
spec_5, phot_5, _ = model.predict(thetas['thetas_5'], obs=obs, sps=sps)
spec_6, phot_6, _ = model.predict(thetas['thetas_6'], obs=obs, sps=sps)

spec_7, phot_7, _   = model.predict(thetas['thetas_7'], obs=obs, sps=sps)
spec_8, phot_8, _   = model.predict(thetas['thetas_8'], obs=obs, sps=sps)
spec_9, phot_9, _   = model.predict(thetas['thetas_9'], obs=obs, sps=sps)
spec_10, phot_10, _ = model.predict(thetas['thetas_10'], obs=obs, sps=sps)
spec_11, phot_11, _ = model.predict(thetas['thetas_11'], obs=obs, sps=sps)
spec_12, phot_12, _ = model.predict(thetas['thetas_12'], obs=obs, sps=sps)


## Change the SNR 
- the snr can not be larger than 20

In [46]:
snr_el_corrected    = []
snr_phot_corrected  = []

for i in snr_el:
    if i<20:
        snr_el_corrected.append(i)
    else:
        snr_el_corrected.append(20)

for i in snr_phot:
    if i<20:
        snr_phot_corrected.append(i)
    else:
        snr_phot_corrected.append(20)

## Generate the Uncertainties

In [47]:
phot_7_dis  = np.random.normal(loc=0., scale=np.abs(phot_7/snr_phot_corrected))
phot_8_dis  = np.random.normal(loc=0., scale=np.abs(phot_8/snr_phot_corrected))
phot_9_dis  = np.random.normal(loc=0., scale=np.abs(phot_9/snr_phot_corrected))
phot_10_dis = np.random.normal(loc=0., scale=np.abs(phot_10/snr_phot_corrected))
phot_11_dis = np.random.normal(loc=0., scale=np.abs(phot_11/snr_phot_corrected))
phot_12_dis = np.random.normal(loc=0., scale=np.abs(phot_12/snr_phot_corrected))

spec_7_dis  = np.random.normal(loc=0., scale=np.abs(spec_7/snr_el_corrected))
spec_8_dis  = np.random.normal(loc=0., scale=np.abs(spec_8/snr_el_corrected))
spec_9_dis  = np.random.normal(loc=0., scale=np.abs(spec_9/snr_el_corrected))
spec_10_dis = np.random.normal(loc=0., scale=np.abs(spec_10/snr_el_corrected))
spec_11_dis = np.random.normal(loc=0., scale=np.abs(spec_11/snr_el_corrected))
spec_12_dis = np.random.normal(loc=0., scale=np.abs(spec_12/snr_el_corrected))


In [48]:
phot_7_unc  =   phot_7/snr_phot_corrected
phot_8_unc  =   phot_8/snr_phot_corrected
phot_9_unc  =   phot_9/snr_phot_corrected
phot_10_unc =   phot_10/snr_phot_corrected
phot_11_unc =   phot_11/snr_phot_corrected
phot_12_unc =   phot_12/snr_phot_corrected

spec_7_unc  =   spec_7/snr_el_corrected
spec_8_unc  =   spec_8/snr_el_corrected
spec_9_unc  =   spec_9/snr_el_corrected
spec_10_unc =   spec_10/snr_el_corrected
spec_11_unc =   spec_11/snr_el_corrected
spec_12_unc =   spec_12/snr_el_corrected

In [49]:
phot_0_dis  = np.random.normal(loc=0., scale=np.abs(phot_0/snr_phot_corrected))
phot_1_dis  = np.random.normal(loc=0., scale=np.abs(phot_1/snr_phot_corrected))
phot_2_dis  = np.random.normal(loc=0., scale=np.abs(phot_2/snr_phot_corrected))
phot_3_dis  = np.random.normal(loc=0., scale=np.abs(phot_3/snr_phot_corrected))
phot_4_dis  = np.random.normal(loc=0., scale=np.abs(phot_4/snr_phot_corrected))
phot_5_dis  = np.random.normal(loc=0., scale=np.abs(phot_5/snr_phot_corrected))
phot_6_dis  = np.random.normal(loc=0., scale=np.abs(phot_6/snr_phot_corrected))

spec_0_dis  = np.random.normal(loc=0., scale=np.abs(spec_0/snr_el_corrected))
spec_1_dis  = np.random.normal(loc=0., scale=np.abs(spec_1/snr_el_corrected))
spec_2_dis  = np.random.normal(loc=0., scale=np.abs(spec_2/snr_el_corrected))
spec_3_dis  = np.random.normal(loc=0., scale=np.abs(spec_3/snr_el_corrected))
spec_4_dis  = np.random.normal(loc=0., scale=np.abs(spec_4/snr_el_corrected))
spec_5_dis  = np.random.normal(loc=0., scale=np.abs(spec_5/snr_el_corrected))
spec_6_dis  = np.random.normal(loc=0., scale=np.abs(spec_6/snr_el_corrected))

In [50]:
phot_0_unc = phot_0/snr_phot_corrected
phot_1_unc = phot_1/snr_phot_corrected
phot_2_unc = phot_2/snr_phot_corrected
phot_3_unc = phot_3/snr_phot_corrected
phot_4_unc = phot_4/snr_phot_corrected
phot_5_unc = phot_5/snr_phot_corrected
phot_6_unc = phot_6/snr_phot_corrected

spec_0_unc = spec_0/snr_el_corrected
spec_1_unc = spec_1/snr_el_corrected
spec_2_unc = spec_2/snr_el_corrected
spec_3_unc = spec_3/snr_el_corrected
spec_4_unc = spec_4/snr_el_corrected
spec_5_unc = spec_5/snr_el_corrected
spec_6_unc = spec_6/snr_el_corrected

## Generate the Data
- combine phot with the generated distortion
- take the distortion as the uncertainty
- set the masks to True everywhere
- take the same data than for the galaxies for
    - filters, wave_effective, wavelength, line_ind, cat_row, id, redshift and line_names

In [51]:
dis_data = {}


index        =  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
phot_all     =  [phot_0, phot_1, phot_2, phot_3, phot_4, phot_5, phot_6, phot_7, phot_8, phot_9, phot_10, phot_11, phot_12]
phot_all_unc =  [phot_0_unc, phot_1_unc, phot_2_unc, phot_3_unc, phot_4_unc, phot_5_unc, phot_6_unc, phot_7_unc , phot_8_unc , phot_9_unc , phot_10_unc, phot_11_unc, phot_12_unc]
phot_all_dis =  [phot_0_dis, phot_1_dis, phot_2_dis, phot_3_dis, phot_4_dis, phot_5_dis, phot_6_dis, phot_7_dis , phot_8_dis , phot_9_dis , phot_10_dis, phot_11_dis, phot_12_dis]
spec_all     =  [spec_0, spec_1, spec_2, spec_3, spec_4, spec_5, spec_6, spec_7, spec_8, spec_9, spec_10, spec_11, spec_12]
spec_all_unc =  [spec_0_unc, spec_1_unc, spec_2_unc, spec_3_unc, spec_4_unc, spec_5_unc, spec_6_unc, spec_7_unc , spec_8_unc , spec_9_unc , spec_10_unc, spec_11_unc, spec_12_unc]
spec_all_dis =  [spec_0_dis, spec_1_dis, spec_2_dis, spec_3_dis, spec_4_dis, spec_5_dis, spec_6_dis, spec_7_dis , spec_8_dis , spec_9_dis , spec_10_dis, spec_11_dis, spec_12_dis]

for i, p in zip(index, range(len(phot_all))):
    dis_data['thetas_'+i]      =   thetas['thetas_'+i]
    dis_data['maggies_'+i]     =   phot_all[p] + phot_all_dis[p]
    dis_data['maggies_unc_'+i] =   phot_all_unc[p]
    dis_data['spec_'+i]        =   spec_all[p] + spec_all_dis[p]
    dis_data['spec_unc_'+i]    =   spec_all_unc[p]

dis_data['filters']         =   obs['filters']
dis_data['wave_effective']  =   obs['wave_effective']
dis_data['wavelength']      =   obs['wavelength']
dis_data['mask']            =   [True for i in dis_data['wavelength']]
dis_data['phot_mask']       =   [True for i in dis_data['filters']]
dis_data['line_ind']        =   obs['line_ind']
dis_data['cat_row']         =   obs['cat_row']
dis_data['id']              =   obs['id']
dis_data['z_spec']          =   obs['z_spec']
dis_data['line_names']      =   obs['line_names']

## Save the Distorted Data in Pickle:

In [52]:
filename = path_data + 'distorted_data'+iot_number+'.pickle'

with open(filename, 'wb') as f:
    pickle.dump(dis_data, f)


The distorted data is now used as the new obs dictionary in the next step, which is putting everything into prospector again. For that we use DISPRO.py.

In [53]:
dis_data

{'thetas_0': [-1.0,
  0.6,
  8.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  -1.0,
  -2.0,
  0.0],
 'maggies_0': array([5.01170851e-12, 1.01926118e-11, 2.86312507e-11, 3.74686711e-11,
        4.81386169e-11, 1.59748817e-12, 3.08153122e-12]),
 'maggies_unc_0': array([9.79026285e-13, 1.83504647e-12, 1.85656910e-12, 1.75917227e-12,
        2.39035607e-12, 7.44934204e-14, 2.98999097e-13]),
 'spec_0': array([2.78892439e-19, 3.95072375e-19, 4.99022267e-19, 8.12142967e-20,
        1.70838358e-19, 1.77574493e-19, 2.96644073e-19, 6.44449828e-19,
        1.28258018e-19, 1.40085714e-18, 1.90046297e-18, 5.55121900e-18,
        2.32306756e-19, 2.15544381e-20, 1.28933312e-20, 6.45115309e-18,
        5.91896219e-20, 1.30536343e-19, 9.52876920e-20]),
 'spec_unc_0': array([1.53541494e-20, 2.11110740e-20, 2.53153527e-20, 7.95720772e-21,
        1.93777502e-20, 2.01685012e-20, 1.91560858e-20, 3.01941656e-20,
        1.78731993e-20, 7.45694977e-20, 9.70449409e-20, 2.95878562e-

In [54]:
dis_data['z_spec']

0.3061555